<img src="images/logodwengo.png" alt="LogoDwengo" style="width:250px;"/>

<div style='color: #690027;' markdown="1">
    <h1>SIMULEER EEN EPIDEMIE: HET SIR-MODEL</h1> 
</div>

<div class="alert alert-box alert-success">
In dit project bestudeer je hoe ziektes zich kunnen verspreiden doorheen een (sociaal) netwerk. Je onderzoekt hoe de structuur van een netwerk een invloed kan hebben op hoe snel een ziekte doorgegeven wordt. Finaal zal je ook verschillende strategieën bekijken om de verspreiding van een ziekte tegen te gaan.
</div>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from scipy.spatial import distance_matrix

## Het SIR-model

Een van de eenvoudigste manieren om ziekteverspreiding in een gemeenschap te modelleren, is aan de hand van het SIR-model. **SIR staat voor *Susceptible* (vatbaar), *Infected* (geïnfecteerd) en *Resistant* (resistent of hersteld), de drie types individuen die in een gemeenschap voorkomen.** <br>
Het SIR-model bestaat uit drie vergelijkingen die de veranderingen van het aantal individuen in een bepaalde groep beschrijven. De variabelen die de toestand beschrijven zijn:

-  $S(t)$: het aantal vatbare individuen op tijdstip $t$;
-  $I(t)$: het aantal geïnfecteerde individuen op tijdstip $t$;
-  $R(t)$: het aantal resistente individuen op tijdstip $t$.

Hierbij is t de tijd in een bepaalde tijdseenheid (de tijdseenheid wordt gekozen afhankelijk van het probleem).

Deze beschrijving maakt een eerste grote vereenvoudiging van de werkelijkheid. Er wordt aangenomen dat elk van deze variabelen een reëel getal is, en dat het aantal individuen in elke groep continu kan variëren. In werkelijkheid zijn het discrete waarden: het aantal geïnfecteerden en vatbare individuen is een natuurlijk getal, je bent immers besmet of je bent het niet. Modelleerders werken echter graag met continue variabelen omdat ze dan de technieken van de wiskundige analyse kunnen gebruiken.

> **Oefening 1**: Onder welke omstandigheden gaat deze continue benadering ongeveer op? Denk je dat je dit model kan gebruiken om een gezin van vier personen te beschrijven?

Deze drie variabelen worden aan elkaar gelinkt aan de hand van drie differentiaalvergelijkingen (die elk een verandering in de tijd beschrijven). Hierbij wordt aangenomen dat de grootte van de populatie ongewijzigd blijft: je neemt dus aan dat, gedurende de tijdspanne die het model beschrijft, er niemand geboren wordt en ook niemand sterft. Eigenlijk beperk je je hier tot de verspreiding van een relatief onschuldige ziekte zoals een verkoudheid. Je kan de situatie dus voorstellen met het volgende stelsel differentiaalvergelijkingen: 
 
$$
\frac{\text{d}S(t)}{\text{d}t} = -\beta \, S(t) \, I(t)
$$

$$
\frac{\text{d}I(t)}{\text{d}t} = \beta \, S(t) \,I(t) - \gamma \, I(t)
$$

$$
\frac{\text{d}R(t)}{\text{d}t} = \gamma \, I(t)
$$

<div class="alert alert-box alert-info">
Elke vergelijking vertelt hoe het aantal mensen in elke groep wijzigt doorheen de tijd. Daaruit kan je berekenen hoeveel mensen zich op een bepaald moment bevinden in elke groep. De parameters $\beta$ en $\gamma$ spelen daarbij een fundamentele rol.
</div>

De vergelijkingen zijn gekoppeld via de *overgangspercentages*. Elk overgangspercentage vertelt hoe van de ene naar de andere groep wordt overgegaan. <br>
Het overgangspercentage van vatbaar (S) naar geïnfecteerd (I) hangt af van het contact tussen een vatbare persoon en een geïnfecteerde persoon. Men noemt dit *infectiepercentage* $\beta$. Dit betekent dat één geïnfecteerde persoon $\beta S$ personen zal besmetten. Het aantal vatbare personen vermindert dus met $\beta S I$ per tijdseenheid. <br>
Het overgangspercentage van geïnfecteerd (I) naar resistent (R) hangt alleen af van het *herstelpercentage*, dat men $\gamma$ noemt. Het aantal geïnfecteerde personen vermindert dus met  $\gamma I$ per tijdseenheid.

> **Oefening 2**: Toon aan met een berekening dat het totaal aantal individuen in de populatie $(S(t)+I(t)+R(t))$ constant zal blijven.

<div class="alert alert-box alert-info">
Het SIR-model is moeilijk om exact te worden opgelost. Dit is het geval bij veel differentiaalvergelijkingen die optreden in de biologische wetenschappen. Je moet dus een <em>numerieke benadering</em> van de oplossing vinden. Dit betekent dat je een algoritme zal gebruiken om een geschatte maar nauwkeurige oplossing te vinden. Vanuit deze oplossing kan je leren hoe de verschillende variabelen in de loop van de tijd veranderen.
</div>

Er zijn verschillende mogelijkheden om dit te doen: 

-  Je zou het continue probleem kunnen vervangen door een **discrete** tegenhanger. <br>
Dit zou je toelaten bepaalde numerieke methoden te gebruiken om een benaderende oplossing te krijgen. 

-  Anderzijds kan je een **iteratieve** methode gebruiken. <br>Uitgaande van een initiële schatting, maken iteratieve methoden opeenvolgende benaderingen die stapsgewijs convergeren naar de exacte oplossing.

## Iteratieve manier

Met behulp van computers is het gemakkelijk om iteratief een numerieke oplossing voor het SIR-model te vinden. <br>
Om dit te doen, vertrek je van een beginvoorwaarde: het is logisch om te beginnen met een populatie met nul resistente personen, een paar geïnfecteerde personen en de rest vatbaar. Vervolgens kan je de numerieke oplossing gebruiken om het aantal mensen in elke groep op bepaalde tijdstippen te berekenen.

Via de Python-module SciPy kan je eenvoudig dergelijke differentiaalvergelijkingen simuleren. Eerst moet je de differentiaalvergelijkingen *implementeren*: je stopt daartoe de drie vergelijkingen hierboven gegeven in een rijmatrix.

Met behulp van de Python-module NumPy kan een matrix voorgesteld worden met een *Numpy array*.

In [ ]:
def SIR(t, y, beta, gamma):
    """Differentiaalvergelijkingen die S, I en R in functie van de tijd t bepalen."""
    S, I, R = y
    return np.array([-beta * S * I,
                    beta * S * I - gamma * I,
                    gamma * I])

Nu kan je het stelsel differentiaalvergelijkingen *numeriek oplossen* met de [solve_ivp](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_ivp.html)-functie voor een bepaalde *beginsituatie*.

### Voorbeeld 1
Beschouw een populatie met 1000 mensen, waarvan initieel één persoon ($I_0$) geïnfecteerd is (en $S_0=999$ personen vatbaar zijn voor de ziekte).

In [ ]:
# voorbeeld 1 
# beginsituatie
S0 = 999
I0 = 1
R0 = 0

beta = 0.001
gamma = 0.1

oplossing = solve_ivp(SIR,                     # functie met parameters
                      [0, 100],                # tijdsinterval waarin we simuleren
                      np.array([S0, I0, R0]),  # initiële omstandigheden
                      args=(beta, gamma))    # parameters van stelsel differentiaalvergelijkingen 

Deze oplossing geef je dan op verschillende manieren grafisch weer:

In [ ]:
# voorbeeld 1 grafiek oplossing S, I, R 
plt.figure()

plt.plot(oplossing.t, oplossing.y[1], color="purple")  # I 
plt.plot(oplossing.t, oplossing.y[2], color="green")   # R
plt.plot(oplossing.t, oplossing.y[0], color="orange")  # S

plt.show()

In [ ]:
# voorbeeld 1 grafiek verdeling populatie over S, I, R in functie van de tijd
plt.figure()

plt.stackplot(oplossing.t, oplossing.y[[1,0,2],:],
              labels=["I", "S", "R"],
              colors=["red", "yellow", "lightgreen"])
plt.xlabel("Tijd")
plt.ylabel("Aantal personen")
plt.legend(loc=0)

plt.show()

In [ ]:
# grafiek voorbeeld 1 combinatie verdeling populatie en S, I, R
plt.figure()

plt.stackplot(oplossing.t, oplossing.y[[1,0,2],:],
              labels=["I", "S", "R"],
              colors=["red", "yellow", "lightgreen"])
plt.xlabel("Tijd")
plt.ylabel("Aantal personen")
plt.legend(loc=0)

plt.plot(oplossing.t, oplossing.y[1], color="purple")  # I 
plt.plot(oplossing.t, oplossing.y[2], color="green")   # R
plt.plot(oplossing.t, oplossing.y[0], color="orange")  # S

plt.show()

> **Oefening 3**: Pas voorbeeld 1 aan en simuleer een aantal situaties door de parameters aan te passen: 
-  Wat als initieel de helft van de populatie resistent was? 
-  Wat als initieel 80 % van de populatie resistent was?

### Voorbeeld 2

In [ ]:
# voorbeeld 2 
# beginsituatie
S0 = 1099
I0 = 1
R0 = 0

beta = 0.0001
gamma = 0.048


oplossing = solve_ivp(SIR,                     # functie met parameters
                      [0, 365],                # tijdsinterval waarin we simuleren
                      np.array([S0, I0, R0]),  # initiële omstandigheden
                      t_eval=np.linspace(0,365,36),   # aantal punten van oplossing
                      args=(beta, gamma))    # parameters van stelsel differentiaalvergelijkingen 

In [ ]:
# voorbeeld 2 grafiek oplossing S, I, R 
plt.figure()

plt.plot(oplossing.t, oplossing.y[1], color="purple")  # I 
plt.plot(oplossing.t, oplossing.y[2], color="green")   # R
plt.plot(oplossing.t, oplossing.y[0], color="orange")  # S

plt.show()

In [ ]:
# grafiek voorbeeld 2 grafiek verdeling populatie over S, I, R in functie van de tijd
plt.figure()

plt.stackplot(oplossing.t, oplossing.y[[1,0,2],:],
              labels=["I", "S", "R"],
              colors=["red", "yellow", "green"])
plt.xlabel("Tijd")
plt.ylabel("Aantal personen")
plt.legend(loc=0)

plt.show()

In [ ]:
# grafiek voorbeeld 2 combinatie verdeling populatie en S, I, R
plt.figure()

plt.stackplot(oplossing.t, oplossing.y[[1,0,2],:],
              labels=["I", "S", "R"],
              colors=["red", "yellow", "lightgreen"])
plt.xlabel("Tijd")
plt.ylabel("Aantal personen")
plt.legend(loc=0)

plt.plot(oplossing.t, oplossing.y[1], color="purple")  # I 
plt.plot(oplossing.t, oplossing.y[2], color="green")   # R
plt.plot(oplossing.t, oplossing.y[0], color="orange")  # S

plt.show()

> **Oefening 4**: Bereken de waarde van $S$ voor het tijdstip waarop $I$ maximaal is. 

> **Oefening 5**: Pas voorbeeld 2 aan door het infectiepercentage $\beta$ met een kwart te verminderen. Hoe verandert de grafiek? 

<img src="images/cclic.png" alt="Banner" align="left" style="width:100px;"/><br><br>
Deze notebook van M. Stock en F. wyffels voor Dwengo vzw is in licentie gegeven volgens een <a href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Naamsvermelding-NietCommercieel-GelijkDelen 4.0 Internationaal-licentie</a>. 